In [20]:
import pandas as pd
import numpy as np

# Random dataset

In [21]:
def random_bool(size):
    return np.random.rand(size) > 0.5

def random_int(min, max, size):
    return np.random.randint(min, max, size=size)

def random_float(min, max, size):
    return np.random.uniform(min, max, size=size)


def index_id(max):
    list = []
    for i in range(1, max):
        list.append('ID' + str(i))
    return list

In [22]:
pre_op = pd.DataFrame({'IDno': index_id(501), 'Død inden for 1 år af operation': random_bool(500), 'age': random_int(18, 89, 500), 'sex': random_bool(500), 'Respiratory Disease': random_bool(500),
       'Circulatory Organ Disease': random_bool(500), 'Type 1 Diabetes': random_bool(500), 'Type 2 Diabetes': random_bool(500),
       'Other metabolic diseases': random_bool(500), 'Other operation': random_bool(500), 'Genital or urine related diseases': random_bool(500), 'Vægt': random_float(35, 190,500), 'KFN': random_bool(500), 'KFX': random_bool(500), 'KFK': random_bool(500), 'KFM': random_bool(500),
       'KFF': random_bool(500), 'KFP': random_bool(500), 'KFC': random_bool(500), 'KFW': random_bool(500), 'KFJ': random_bool(500), 'Hæmoglobin': random_float(4,12,500), 'Leukocytter': random_float(1, 125, 500),
       'Trombocytter': random_float(20, 1000, 500)})
pre_op = pre_op.replace(True, 1).replace(False, 0)

# Predicting Mortality with pre_op data

## LR

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( pre_op.loc[:, pre_op.columns != 'Død inden for 1 år af operation'], pre_op['Død inden for 1 år af operation'], test_size=0.3, random_state=42)

def lr_pred(X_train, y_train, X_test):
    clf = LogisticRegression().fit(X_train, y_train)
    return clf.predict_proba(X_test)

## Random forrest

## Gradient boost


In [29]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc


def random_bool(size):
    return np.random.rand(size) > 0.5

def random_int(min, max, size):
    return np.random.randint(min, max, size=size)

def random_float(min, max, size):
    return np.random.uniform(min, max, size=size)

def index_id(max):
    return ['ID' + str(i) for i in range(1, max)]

pre_op = pd.DataFrame({
    'IDno': index_id(501),
    'Død inden for 1 år af operation': random_bool(500),
    'age': random_int(18, 89, 500),
    'sex': random_bool(500),
    'Respiratory Disease': random_bool(500),
    'Circulatory Organ Disease': random_bool(500),
    'Type 1 Diabetes': random_bool(500),
    'Type 2 Diabetes': random_bool(500),
    'Other metabolic diseases': random_bool(500),
    'Other operation': random_bool(500),
    'Genital or urine related diseases': random_bool(500),
    'Vægt': random_float(35, 190,500),
    'KFN': random_bool(500),
    'KFX': random_bool(500),
    'KFK': random_bool(500),
    'KFM': random_bool(500),
    'KFF': random_bool(500),
    'KFP': random_bool(500),
    'KFC': random_bool(500),
    'KFW': random_bool(500),
    'KFJ': random_bool(500),
    'Hæmoglobin': random_float(4,12,500),
    'Leukocytter': random_float(1, 125, 500),
    'Trombocytter': random_float(20, 1000, 500)
})

pre_op = pre_op.replace(True, 1).replace(False, 0)

class GradientBoosting:
    def __init__(self, balanced=False):
        self.balanced = balanced
        self.feature_importance = None
        self.shap_values = None
        self.num_trials = 0

    def create_model(self, random_state):
        seed_value = random_state.get_state()[1][0]
        param = {
            "max_depth": 2,
            "learning_rate": 1,  # 'eta' is renamed to 'learning_rate' in XGBClassifier
            "reg_lambda": 0.25,  # 'lambda' is renamed to 'reg_lambda' in XGBClassifier
            "subsample": 0.5,
            "n_jobs": 6,  # 'nthread' is renamed to 'n_jobs' in XGBClassifier
            "objective": "binary:logistic",
            "random_state": seed_value,
        }
        model = XGBClassifier(**param)
        return {
            "model": model,
            "param": param,
            "num_round": 10,  # Not used directly with XGBClassifier
        }

    def fit_model(self, model, train_data, train_labels):
        model["model"].fit(train_data, train_labels)

    def predict_model(self, model, test_data):
        preds = model["model"].predict_proba(test_data)[:, 1]
        return preds

    @staticmethod
    def save_model(model, path):
        bst = model["model"]
        bst.save_model(path)

# Drop the 'IDno' column from the dataset before splitting
pre_op = pre_op.drop(columns=['IDno'])

X_train, X_test, y_train, y_test = train_test_split(pre_op.loc[:, pre_op.columns != 'Død inden for 1 år af operation'], pre_op['Død inden for 1 år af operation'], test_size=0.3, random_state=42)

gb_model = GradientBoosting()
model = gb_model.create_model(np.random.RandomState(42))

gb_model.fit_model(model, X_train, y_train)

prediction = gb_model.predict_model(model, X_test)

# Get probability predictions
prob_predictions = gb_model.predict_model(model, X_test)

# Get binary predictions
binary_predictions = (prob_predictions > 0.5).astype(int)

# Calculate ROC AUC
roc_auc = roc_auc_score(y_test, prob_predictions)
print("ROC AUC Score:", roc_auc)

#Calculate PR AUC
precision, recall, _ = precision_recall_curve(y_test, prob_predictions)
pr_auc = auc(recall, precision)
print("PR AUC Score:", pr_auc)

ROC AUC Score: 0.4734299516908213
PR AUC Score: 0.4267304385325793


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Define the target and features
target = 'Død inden for 1 år af operation'
features = [col for col in pre_op.columns if col != target and col != 'IDno']

X = pre_op[features]
y = pre_op[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class GradientBoosting:
    def __init__(self, balanced=False):
        self.balanced = balanced
        self.model = None

    def create_model(self, random_state):
        params = {
            "max_depth": 2,
            "learning_rate": 1,
            "n_estimators": 10,
            "subsample": 0.5,
            "random_state": random_state.get_state()[1][0],
        }
        if self.balanced:
            params["scale_pos_weight"] = "balanced"
        self.model = GradientBoostingClassifier(**params)
        return self.model

    def fit_model(self, model, train_data, train_labels):
        model.fit(train_data, train_labels)

    def predict_model(self, model, test_data):
        preds = model.predict_proba(test_data)[:, 1]
        return preds

    @staticmethod
    def save_model(model, path):
        import joblib
        joblib.dump(model, path)

# Initialize the model
gb_model = GradientBoosting()

# Create the model parameters
model = gb_model.create_model(np.random.RandomState(42))

# Train the model
gb_model.fit_model(model, X_train, y_train)

# Make predictions on the test set
predictions = gb_model.predict_model(model, X_test)

# Convert probabilities to binary outcomes
threshold = 0.5
binary_predictions = (predictions >= threshold).astype(int)

# Convert the probabilities to a numpy array and print it
probabilities_array = np.array(predictions)
print(probabilities_array)

# Evaluate the binary predictions
accuracy = accuracy_score(y_test, binary_predictions)
conf_matrix = confusion_matrix(y_test, binary_predictions)
class_report = classification_report(y_test, binary_predictions)

[1.92096414e-01 3.12697738e-02 1.16537304e-01 9.55109681e-01
 5.48931985e-01 4.69406146e-01 5.29897849e-03 1.32789905e-01
 2.06973872e-02 2.76404122e-02 3.12531150e-01 6.13207886e-01
 4.30207174e-01 4.69406146e-01 5.24183843e-01 8.34579904e-01
 2.90851368e-01 4.50383812e-01 3.12531150e-01 7.21054260e-01
 3.73593595e-01 8.87907221e-01 2.46339809e-01 9.46731642e-01
 1.92096414e-01 4.15213588e-02 6.07208919e-02 2.59034854e-01
 9.00014501e-02 6.02071253e-01 2.64124153e-01 2.59034854e-01
 7.96009215e-01 4.16343072e-01 8.20316879e-01 9.99665456e-01
 7.57770038e-01 1.14387048e-01 4.69406146e-01 1.32789905e-01
 7.96009215e-01 6.91869151e-01 1.32789905e-01 2.79277037e-01
 9.88444774e-01 3.26554597e-01 8.99081283e-01 5.49772440e-01
 2.83164873e-01 5.29362371e-01 4.69406146e-01 5.89078808e-01
 1.15394182e-04 3.55583992e-01 1.25985527e-03 4.50383812e-01
 9.93857201e-01 7.96009215e-01 6.43372802e-01 8.93665587e-01
 9.12457565e-01 9.19046924e-01 7.35088835e-01 9.87957947e-02
 6.74660954e-01 4.694061

In [40]:
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Accuracy: 0.58
Confusion Matrix:
 [[30 14]
 [28 28]]
Classification Report:
               precision    recall  f1-score   support

           0       0.52      0.68      0.59        44
           1       0.67      0.50      0.57        56

    accuracy                           0.58       100
   macro avg       0.59      0.59      0.58       100
weighted avg       0.60      0.58      0.58       100



## MLP

# Grouped cross-validation

In [212]:
from sklearn.model_selection import GroupKFold

X = pre_op.loc[:, pre_op.columns != 'Død inden for 1 år af operation']
y = pre_op['Død inden for 1 år af operation']

outer_cv = GroupKFold(n_splits = 5)
inner_cv = GroupKFold(n_splits = 5)

for outer_train_idx, outer_test_idx in outer_cv.split(X, y, groups = pre_op['IDno']):
    X_outer_train, y_outer_train = X.loc[outer_train_idx], y.loc[outer_train_idx]
    X_outer_test, y_outer_test = X.loc[outer_test_idx], y.loc[outer_test_idx]

    for inner_train_idx, inner_test_idx in inner_cv.split(X_outer_train, y_outer_train, groups=pre_op['IDno']):
        X_inner_train, y_inner_train = X_outer_train.loc[inner_train_idx], y_outer_train.loc[inner_train_idx]
        X_inner_test, y_inner_test = X_outer_train.loc[inner_test_idx], y_outer_train.loc[inner_test_idx]

        print('works')

ValueError: Found input variables with inconsistent numbers of samples: [400, 400, 500]